#E2E Deep Learning Project Task (Pytorch)

*  **Network used**: Convolutional Neural Network

*  **Frameworks used:** 
Pytorch (for Training)
Scikit-learn (for reporting, data splitting and measuring accuracy)

*  **Environment used:** 
Google Colaboratory
---



Importing libraries

In [0]:
import numpy as np
import torch
from torch.nn.functional import cross_entropy, nll_loss
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.utils.data.distributed
import torchvision.transforms as transforms
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

Setting the paths for the data to be retrieved.

*You can change the variable "pth" according to your data path*

In [0]:
pth= '/content/drive/My Drive/task_GSoC/'   #enter your path here

#Load dataset

photon_data  = h5py.File(pth+'SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5')
photon_X  = np.array(photon_data['X'])
photon_Y  = np.array(photon_data['y'])

electron_data= h5py.File(pth+'SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5')
electron_X= np.array(electron_data['X'])
electron_Y= np.array(electron_data['y'])

X = np.transpose(np.concatenate((photon_X, electron_X)),(0, 3, 1 ,2)) #reshape data to torch format
y = np.concatenate((photon_Y, electron_Y))

In [0]:
#shuffle and train-val-test split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.4)
X_val, X_test, y_val, y_test= train_test_split(X_test, y_test, test_size= 0.5)



In [5]:
X_train.shape

(298800, 2, 32, 32)

##Neural Network

I used the following architecture for the neural network:


*   2d Conv layer (8 3x3 filters with 'same' padding) 
*   2d Conv layer (16 3x3 filters with 'same' padding)
*   2d Avg Pool layer (2x2 filter size)
*   2d Conv layer (32 3x3 filters with 'same' padding)
*   2d Conv layer (48 3x3 filters with 'same' padding)
*   2d Avg Pool layer (2x2 filter size)
*   2d Conv layer (54 3x3 filters with 'same' padding)
*   2d Avg Pool layer (2x2 filter size)
*   2d Conv layer (64 3x3 filters with 'same' padding)
*   2d Avg Pool layer (2x2 filter size)
*   Flatten layer
*   Dense layer with 1024 outputs
*   Dense layer with 512 outputs
*   Dense layer with 64 outputs
*   Dense layer with 32 outputs
*   Output layer
















In [0]:
class CNN(torch.nn.Module):
    
    #Our batch shape for input x is (2, 32, 32)
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(2, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(32, 48, kernel_size=3, stride=1, padding=1)
        self.conv5 = torch.nn.Conv2d(48, 54, kernel_size=3, stride=1, padding=1)
        self.conv6 = torch.nn.Conv2d(54, 64, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(2*2*64, 1024)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(1024, 512)
        self.fc3 = torch.nn.Linear(512, 64)
        self.fc4 = torch.nn.Linear(64, 32)
        self.fc5 = torch.nn.Linear(32, 1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)

        x = F.relu(self.conv5(x))
        x = self.pool(x)
        x = F.relu(self.conv6(x))
        x = self.pool(x)

        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return(x)

**Making a torch Dataset class and generating dataloaders**

In [0]:
#class for the dataset
class MyDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).float()
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

#batch size
bs = 512

#Training Data
train_data = MyDataset(X_train, y_train)
train_loader = DataLoader(
    train_data,
    batch_size = bs,
    shuffle = True,
)

#Validation Data
val_data = MyDataset(X_val, y_val)
val_loader = DataLoader(
    val_data,
    batch_size = bs,
    shuffle = True,
)

#Test Data
test_data = MyDataset(X_test, y_test)
test_loader = DataLoader(
    test_data,
    batch_size = bs,
    shuffle = True,
)


In [0]:
# defining the model
model = CNN()
model.cuda()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = torch.nn.BCELoss()

In [0]:
# epoch to start from
epochs = 0

Run this block in order to load a pre trained model

In [0]:
#change the name of the .pth file as per requirement
checkpoint = torch.load(pth+"epoch_249.pth")
model.load_state_dict(checkpoint['model'])
epochs = checkpoint['epoch']

**The Training Function**


In [0]:
def train(epoch):
    # to put the model in training mode
    model.train()
    tr_acc = 0
    tr_loss = 0

    #lists to save the progress
    train_accuracies=[]
    train_losses=[]
    val_accuracies=[]
    
    for tr_batch_id, (tr_data, tr_target) in enumerate(train_loader):
      tr_data = tr_data.to('cuda')
      tr_target = tr_target.to('cuda')

      optimizer.zero_grad()
      tr_preds = model(tr_data)
      loss = criterion(tr_preds, tr_target)
      tr_loss += loss
      loss.backward()
      optimizer.step()
      
      #calculating accuracy using sklearn
      binary_preds = (tr_preds>0.5).type(torch.FloatTensor)
      acc = accuracy_score(tr_target.cpu(),binary_preds.cpu())
      tr_acc += acc

      ###### uncomment this to view mini batch progress ######
      # if tr_batch_id % 75 == 0:
      #   print("Batch # {:>3} : Loss = {}".format(tr_batch_id,loss.item()))
      
    tr_acc = 100*(tr_acc / (tr_batch_id+1))
    tr_loss = tr_loss / (tr_batch_id+1)

    # to put the model in evaluation mode
    model.eval() # set model in inference mode (need this because of dropout)
    val_acc = 0
    
    for val_batch_id, (val_data, val_target) in enumerate(val_loader):
        val_data = val_data.to('cuda')
        val_target = val_target.to('cuda')

        val_preds = model(val_data)
        binary_preds = (val_preds>0.5).type(torch.FloatTensor)

        acc = accuracy_score(val_target.cpu(), binary_preds.cpu())
        val_acc += acc

    val_acc = 100*(val_acc / (val_batch_id+1))

    train_accuracies.append(tr_acc)
    val_accuracies.append(val_acc)
    train_losses.append(tr_loss)

    print('Epoch: {}  Train_Loss: ({:.5f})  Train_Accuracy: ({:.3f}%)  Validation_Accuracy: ({:.3f}%)'.format(epoch+1,tr_loss, tr_acc, val_acc))
    # return preds,target

In [27]:
# defining the number of epochs
n_epochs = 255

for epoch in range(epochs,n_epochs):
    train(epoch)

Epoch: 250  Train_Loss: (0.53787)  Train_Accuracy: (73.465%)  Validation_Accuracy: (73.687%)
Epoch: 251  Train_Loss: (0.53797)  Train_Accuracy: (73.462%)  Validation_Accuracy: (73.780%)
Epoch: 252  Train_Loss: (0.53706)  Train_Accuracy: (73.575%)  Validation_Accuracy: (73.873%)
Epoch: 253  Train_Loss: (0.53708)  Train_Accuracy: (73.495%)  Validation_Accuracy: (73.801%)
Epoch: 254  Train_Loss: (0.53672)  Train_Accuracy: (73.562%)  Validation_Accuracy: (73.810%)
Epoch: 255  Train_Loss: (0.53689)  Train_Accuracy: (73.533%)  Validation_Accuracy: (73.819%)


**Saving the model**

In [0]:
state = {'model': model.state_dict(),
         'epoch': epoch}
torch.save(state,pth+'epoch_'+str(epoch)+'.pth')

**Testing the Model**

In [0]:
def test():
  model.eval()
  test_acc=0

  for test_batch_id, (test_data, test_target) in enumerate(test_loader):
        test_data = test_data.to('cuda')
        test_target = test_target.to('cuda')

        test_preds = model(test_data)
        binary_preds = (test_preds>0.5).type(torch.FloatTensor)

        acc = accuracy_score(test_target.cpu(), binary_preds.cpu())
        test_acc += acc

  test_acc = 100*(test_acc / (test_batch_id+1))
  
  print('Test_Accuracy: ({:.3f}%)'.format(test_acc))

In [30]:
test()

Test_Accuracy: (73.494%)
